In [16]:
import os
import glob
import pandas as pd
import tensorflow as tf
import numpy as np
import json
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB2

In [17]:
base_folder = 'G:/내 드라이브/colab/data/test/dog/'
class_folders = os.listdir(base_folder)

In [18]:
image_paths = []
json_paths = []

for class_folder in class_folders:
    image_files = glob.glob(os.path.join(base_folder, class_folder, '*.jpg'))
    json_files = [file.replace('.jpg', '.json') for file in image_files]

    image_paths.extend(image_files)
    json_paths.extend(json_files)

image_paths = [path.replace('\\', '/') for path in image_paths]
json_paths = [path.replace('\\', '/') for path in json_paths]

df = pd.DataFrame({
    'image_path': image_paths,
    'json_path': json_paths,
    'label': [path.split('/')[-2] for path in image_paths]
})

In [19]:
df['image_path'][0]

'G:/내 드라이브/colab/data/test/dog/A1/IMG_D_A1_004832.jpg'

In [20]:
def create_mask(image_path, json_path):  
    try:        
        with Image.open(image_path) as img:
            original_size = img.size
            img = img.resize((260, 260))
            image = np.array(img, dtype=np.float32)

        with open(json_path, 'r', encoding='UTF-8') as file:
            data = json.load(file)

        mask = Image.new('L', original_size, 0)
        draw = ImageDraw.Draw(mask)

        for annotation in data['labelingInfo']:
            if 'polygon' in annotation:
                polygon = []
                for i in range(1, len(annotation['polygon']['location'][0]) // 2 + 1):
                    x_key = f'x{i}'
                    y_key = f'y{i}'
                    x = annotation['polygon']['location'][0].get(x_key)
                    y = annotation['polygon']['location'][0].get(y_key)
                    if x is not None and y is not None:
                        polygon.append((x, y))

                if polygon:
                    draw.polygon(polygon, outline=1, fill=1)

        mask = mask.resize((260, 260))
        mask = np.array(mask, dtype=np.float32)
        mask = np.expand_dims(mask, axis=-1)

        return image, mask
    except Exception as e:
        empty_image = np.zeros((260, 260, 3), dtype=np.float32)
        empty_mask = np.zeros((260, 260, 1), dtype=np.float32)
        return empty_image, empty_mask

def create_masked_image(image_path, json_path):
    masked_image = tf.where(mask > 0, image, tf.zeros_like(image))
    return masked_image

def map_labels(label):
    mapping = {'A1': 0, 'A2': 1, 'A3': 2, 'A4': 3, 'A5': 4, 'A6': 5}
    return mapping[label]

def prepare_dataset(image_path, json_path, label):
    # 마스크가 적용된 이미지 생성
    image, mask = create_mask(image_path, json_path)
    masked_image = create_masked_image(image, mask)
    # 라벨을 정수 인덱스로 변환
    label_index = map_labels(label)
    # 라벨을 원-핫 인코딩으로 변환
    one_hot_label = tf.one_hot(label_index, depth=6)
    return masked_image, one_hot_label

In [21]:
def build_efficientnet_model(num_classes):
    base_model = EfficientNetB2(weights='imagenet', include_top=False, input_shape=(260, 260, 3))
    base_model.trainable = False  # Transfer learning: base model is not trainable

    # Add custom layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    # Final model
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [22]:
# 모델 생성
efficientnet_model = build_efficientnet_model(num_classes=6)

# 콜백 설정
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, verbose=1),
    ModelCheckpoint('Efficient_best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)
]

# 데이터셋 생성 및 전처리
dataset = tf.data.Dataset.from_tensor_slices((df['image_path'], df['json_path'], df['label']))
dataset = dataset.map(prepare_dataset, num_parallel_calls=tf.data.AUTOTUNE)

# 데이터셋 분할: 훈련과 검증
train_size = int(0.8 * len(df))
val_size = len(df) - train_size
train_dataset = dataset.take(train_size).batch(32).prefetch(tf.data.AUTOTUNE)
val_dataset = dataset.skip(train_size).batch(32).prefetch(tf.data.AUTOTUNE)

Exception: URL fetch failure on https://storage.googleapis.com/keras-applications/efficientnetb2_notop.h5: None -- unknown url type: https

In [ ]:
history = efficientnet_model.fit(
    train_dataset,
    epochs=20,
    validation_data=val_dataset,
    callbacks=callbacks
)